# Database migration rules

In [1]:
import fludashboard as flud
import pandas as pd
import os
import glob

In [2]:
# pandas configuration
pd.set_option('display.max_columns', 99)

In [3]:
def get_filename_from_path(file_path: str):
    """
    """
    return file_path.split(os.path.sep)[-1].split('.')[0]

In [4]:
path_data_files = os.path.join(flud.__path__[0], 'data', '*.csv')

In [5]:
dfs = {}
pks = {}

In [6]:
print('Data files:')
for file_path in glob.glob(path_data_files):
    filename = get_filename_from_path(file_path)
    
    print(filename)
    
    dfs[filename] = pd.read_csv(file_path)

Data files:
mem-report
clean_data_epiweek-weekly-incidence_w_situation


/home/marfcg/miniconda3/envs/fludashboard-development/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


historical_estimated_values
current_estimated_values
mem-typical


# Setting IDs

## Datasets

In [7]:
dataset_id = {
    'srag': 1,
    'sragflu': 2,
    'obitoflu': 3,
}

## Scale

In [8]:
scale_id = {
    'incidência': 1,
    'casos': 2
}

## current_estimated_values

In [9]:
dataset = 'current_estimated_values'

In [10]:
dfs[dataset].head()

UF  epiyear  epiweek  SRAG    Tipo Situation  mean  50%  2.5%  97.5%   L0  \
0  11     2009        1   0.0  Estado    stable   0.0  0.0   0.0    0.0  1.0   
1  11     2009        2   0.0  Estado    stable   0.0  0.0   0.0    0.0  1.0   
2  11     2009        3   0.0  Estado    stable   0.0  0.0   0.0    0.0  1.0   
3  11     2009        4   0.0  Estado    stable   0.0  0.0   0.0    0.0  1.0   
4  11     2009        5   0.0  Estado    stable   0.0  0.0   0.0    0.0  1.0   

    L1   L2   L3    Run date  dado      escala  
0  0.0  0.0  0.0  2017-10-11  srag  incidência  
1  0.0  0.0  0.0  2017-10-11  srag  incidência  
2  0.0  0.0  0.0  2017-10-11  srag  incidência  
3  0.0  0.0  0.0  2017-10-11  srag  incidência  
4  0.0  0.0  0.0  2017-10-11  srag  incidência

In [11]:
migration_rules = {
    'UF': 'state_country',
    'SRAG': 'value',
    'Tipo': 'type',
    'Situation': 'situation',
    '50%': 'median',
    '2.5%': 'ci_lower',
    '97.5%': 'ci_upper',
    'L0': 'low',
    'L1': 'epidemic',
    'L2': 'high',
    'L3': 'very_high',
    'Run date': 'run_date',
    'dado': 'dataset',
    'escala': 'scale'
}

dfs[dataset].rename(
    columns=migration_rules, inplace=True
)
dfs[dataset].dataset = dfs[dataset].dataset.map(dataset_id)
dfs[dataset].scale = dfs[dataset].scale.map(scale_id)
dfs[dataset].head()

state_country  epiyear  epiweek  value    type situation  mean  median  \
0            11     2009        1    0.0  Estado    stable   0.0     0.0   
1            11     2009        2    0.0  Estado    stable   0.0     0.0   
2            11     2009        3    0.0  Estado    stable   0.0     0.0   
3            11     2009        4    0.0  Estado    stable   0.0     0.0   
4            11     2009        5    0.0  Estado    stable   0.0     0.0   

   ci_lower  ci_upper  low  epidemic  high  very_high    run_date  dataset  \
0       0.0       0.0  1.0       0.0   0.0        0.0  2017-10-11        1   
1       0.0       0.0  1.0       0.0   0.0        0.0  2017-10-11        1   
2       0.0       0.0  1.0       0.0   0.0        0.0  2017-10-11        1   
3       0.0       0.0  1.0       0.0   0.0        0.0  2017-10-11        1   
4       0.0       0.0  1.0       0.0   0.0        0.0  2017-10-11        1   

   scale  
0      1  
1      1  
2      1  
3      1  
4      1

In [12]:
# primary_keys
pks[dataset] = ['dataset', 'scale', 'state_country', 'epiyear', 'epiweek']

dfs[dataset].set_index(pks[dataset], inplace=True)
dfs[dataset].head()

value    type situation  mean  \
dataset scale state_country epiyear epiweek                                  
1       1     11            2009    1          0.0  Estado    stable   0.0   
                                    2          0.0  Estado    stable   0.0   
                                    3          0.0  Estado    stable   0.0   
                                    4          0.0  Estado    stable   0.0   
                                    5          0.0  Estado    stable   0.0   

                                             median  ci_lower  ci_upper  low  \
dataset scale state_country epiyear epiweek                                    
1       1     11            2009    1           0.0       0.0       0.0  1.0   
                                    2           0.0       0.0       0.0  1.0   
                                    3           0.0       0.0       0.0  1.0   
                                    4           0.0       0.0       0.0  1.0   
                                    5           0.0       0.0       0.0  1.0   

                                             epidemic  high  very_high  \
dataset scale state_country epiyear epiweek                              
1       1     11            2009    1             0.0   0.0        0.0   
                                    2             0.0   0.0        0.0   
                                    3             0.0   0.0        0.0   
                                    4             0.0   0.0        0.0   
                                    5             0.0   0.0        0.0   

                                               run_date  
dataset scale state_country epiyear epiweek              
1       1     11            2009    1        2017-10-11  
                                    2        2017-10-11  
                                    3        2017-10-11  
                                    4        2017-10-11  
                                    5        2017-10-11

## historical_estimated_values

In [13]:
dataset = 'historical_estimated_values'

In [14]:
dfs[dataset].head()

UF  epiyear  epiweek      SRAG    Tipo  Situation      mean       50%  \
0  11     2017        7  0.055377  Estado  estimated  0.055377  0.055377   
1  11     2017        8  0.055377  Estado  estimated  0.055377  0.055377   
2  11     2017        9  0.000000  Estado  estimated  0.000000  0.000000   
3  11     2017       10  0.000000  Estado  estimated  0.000000  0.000000   
4  11     2017       11  0.055377  Estado  estimated  0.055377  0.055377   

       2.5%     97.5%     L0     L1   L2   L3    Run date base_epiyearweek  \
0  0.055377  0.055377  0.976  0.024  0.0  0.0  2017-10-11          2017W23   
1  0.055377  0.110755  0.972  0.028  0.0  0.0  2017-10-11          2017W23   
2  0.000000  0.000000  1.000  0.000  0.0  0.0  2017-10-11          2017W23   
3  0.000000  0.055377  0.996  0.004  0.0  0.0  2017-10-11          2017W23   
4  0.055377  0.110755  0.964  0.036  0.0  0.0  2017-10-11          2017W23   

   base_epiyear  base_epiweek  dado      escala  
0          2017            23  srag  incidência  
1          2017            23  srag  incidência  
2          2017            23  srag  incidência  
3          2017            23  srag  incidência  
4          2017            23  srag  incidência

In [15]:
migration_rules = {
    'UF': 'state_country',
    'SRAG': 'value',
    'Tipo': 'type',
    'Situation': 'situation',
    '50%': 'median',
    '2.5%': 'ci_lower',
    '97.5%': 'ci_upper',
    'L0': 'low',
    'L1': 'epidemic',
    'L2': 'high',
    'L3': 'very_high',
    'Run date': 'run_date',
    'dado': 'dataset',  # or origin
    'escala': 'scale'
}

dfs[dataset].rename(
    columns=migration_rules, inplace=True
)
dfs[dataset].dataset = dfs[dataset].dataset.map(dataset_id)
dfs[dataset].scale = dfs[dataset].scale.map(scale_id)
dfs[dataset].head()

state_country  epiyear  epiweek     value    type  situation      mean  \
0            11     2017        7  0.055377  Estado  estimated  0.055377   
1            11     2017        8  0.055377  Estado  estimated  0.055377   
2            11     2017        9  0.000000  Estado  estimated  0.000000   
3            11     2017       10  0.000000  Estado  estimated  0.000000   
4            11     2017       11  0.055377  Estado  estimated  0.055377   

     median  ci_lower  ci_upper    low  epidemic  high  very_high    run_date  \
0  0.055377  0.055377  0.055377  0.976     0.024   0.0        0.0  2017-10-11   
1  0.055377  0.055377  0.110755  0.972     0.028   0.0        0.0  2017-10-11   
2  0.000000  0.000000  0.000000  1.000     0.000   0.0        0.0  2017-10-11   
3  0.000000  0.000000  0.055377  0.996     0.004   0.0        0.0  2017-10-11   
4  0.055377  0.055377  0.110755  0.964     0.036   0.0        0.0  2017-10-11   

  base_epiyearweek  base_epiyear  base_epiweek  dataset  scale  
0          2017W23          2017            23        1      1  
1          2017W23          2017            23        1      1  
2          2017W23          2017            23        1      1  
3          2017W23          2017            23        1      1  
4          2017W23          2017            23        1      1

In [16]:
# primary_keys
pks[dataset] = [
    'dataset', 'scale', 'state_country', 
    'base_epiyear', 'base_epiweek', 
    'epiyear', 'epiweek'
]

dfs[dataset].set_index(pks[dataset], inplace=True)
dfs[dataset].head()

value  \
dataset scale state_country base_epiyear base_epiweek epiyear epiweek             
1       1     11            2017         23           2017    7        0.055377   
                                                              8        0.055377   
                                                              9        0.000000   
                                                              10       0.000000   
                                                              11       0.055377   

                                                                         type  \
dataset scale state_country base_epiyear base_epiweek epiyear epiweek           
1       1     11            2017         23           2017    7        Estado   
                                                              8        Estado   
                                                              9        Estado   
                                                              10       Estado   
                                                              11       Estado   

                                                                       situation  \
dataset scale state_country base_epiyear base_epiweek epiyear epiweek              
1       1     11            2017         23           2017    7        estimated   
                                                              8        estimated   
                                                              9        estimated   
                                                              10       estimated   
                                                              11       estimated   

                                                                           mean  \
dataset scale state_country base_epiyear base_epiweek epiyear epiweek             
1       1     11            2017         23           2017    7        0.055377   
                                                              8        0.055377   
                                                              9        0.000000   
                                                              10       0.000000   
                                                              11       0.055377   

                                                                         median  \
dataset scale state_country base_epiyear base_epiweek epiyear epiweek             
1       1     11            2017         23           2017    7        0.055377   
                                                              8        0.055377   
                                                              9        0.000000   
                                                              10       0.000000   
                                                              11       0.055377   

                                                                       ci_lower  \
dataset scale state_country base_epiyear base_epiweek epiyear epiweek             
1       1     11            2017         23           2017    7        0.055377   
                                                              8        0.055377   
                                                              9        0.000000   
                                                              10       0.000000   
                                                              11       0.055377   

                                                                       ci_upper  \
dataset scale state_country base_epiyear base_epiweek epiyear epiweek             
1       1     11            2017         23           2017    7        0.055377   
                                                              8        0.110755   
                                                              9        0.000000   
                                                              10       0.055377   
                                                              11       0.110755   

       

## clean_data_epiweek-weekly-incidence_w_situation

In [17]:
dataset = 'clean_data_epiweek-weekly-incidence_w_situation'

In [18]:
dfs[dataset].head()

0-4 anos  10-19 anos  2-4 anos  20-29 anos  30-39 anos  40-49 anos  \
0       0.0         0.0       NaN         0.0         0.0         0.0   
1       0.0         0.0       NaN         0.0         0.0         0.0   
2       0.0         0.0       NaN         0.0         0.0         0.0   
3       0.0         0.0       NaN         0.0         0.0         0.0   
4       0.0         0.0       NaN         0.0         0.0         0.0   

   5-9 anos  50-59 anos  60+ anos  < 2 anos  DELAYED  FLU_A  FLU_B  \
0       0.0         0.0       0.0       NaN      0.0    0.0    0.0   
1       0.0         0.0       0.0       NaN      0.0    0.0    0.0   
2       0.0         0.0       0.0       NaN      0.0    0.0    0.0   
3       0.0         0.0       0.0       NaN      0.0    0.0    0.0   
4       0.0         0.0       0.0       NaN      0.0    0.0    0.0   

   INCONCLUSIVE  Idade desconhecida  NEGATIVE  NOTTESTED  OTHERS  \
0           0.0                 NaN       0.0        0.0     0.0   
1           0.0                 NaN       0.0        0.0     0.0   
2           0.0                 NaN       0.0        0.0     0.0   
3           0.0                 NaN       0.0        0.0     0.0   
4           0.0                 NaN       0.0        0.0     0.0   

   POSITIVE_CASES  SRAG Situation  TESTING_IGNORED    Tipo  UF  \
0             0.0   0.0    stable              0.0  Estado  11   
1             0.0   0.0    stable              0.0  Estado  11   
2             0.0   0.0    stable              0.0  Estado  11   
3             0.0   0.0    stable              0.0  Estado  11   
4             0.0   0.0    stable              0.0  Estado  11   

  Unidade da Federação  VSR  dado  epiweek  epiyear epiyearweek      escala  \
0             Rondônia  0.0  srag        1     2009     2009W01  incidência   
1             Rondônia  0.0  srag        1     2009     2009W01  incidência   
2             Rondônia  0.0  srag        1     2009     2009W01  incidência   
3             Rondônia  0.0  srag        2     2009     2009W02  incidência   
4             Rondônia  0.0  srag        2     2009     2009W02  incidência   

    sexo  
0      F  
1      M  
2  Total  
3      F  
4      M

In [19]:
dfs[dataset].keys()

Index(['0-4 anos', '10-19 anos', '2-4 anos', '20-29 anos', '30-39 anos',
       '40-49 anos', '5-9 anos', '50-59 anos', '60+ anos', '< 2 anos',
       'DELAYED', 'FLU_A', 'FLU_B', 'INCONCLUSIVE', 'Idade desconhecida',
       'NEGATIVE', 'NOTTESTED', 'OTHERS', 'POSITIVE_CASES', 'SRAG',
       'Situation', 'TESTING_IGNORED', 'Tipo', 'UF', 'Unidade da Federação',
       'VSR', 'dado', 'epiweek', 'epiyear', 'epiyearweek', 'escala', 'sexo'],
      dtype='object')

In [20]:
migration_rules = {
    '0-4 anos': '0_4_years',
    '10-19 anos': '10_19_years',
    '2-4 anos': '2_4_years',
    '20-29 anos': '20_29_years',
    '30-39 anos': '30_39_years',
    '40-49 anos': '40_49_years',
    '5-9 anos': '5_9_years',
    '50-59 anos': '50_59_years',
    '60+ anos': '60_years_or_more',
    '< 2 anos': 'lt_2_years',
    'DELAYED': 'delayed',
    'FLU_A': 'flu_a',
    'FLU_B': 'flu_b',
    'INCONCLUSIVE': 'inconclusive',
    'Idade desconhecida': 'unknown_age',
    'NEGATIVE': 'negative',
    'NOTTESTED': 'not_tested',
    'OTHERS': 'others',
    'POSITIVE_CASES': 'positive_cases',
    'SRAG': 'value',
    'Situation': 'situation',
    'Tipo': 'type',
    'UF': 'state_country',
    'Unidade da Federação': 'state_country_name', # Not needed in the table
    'VSR': 'vsr',
    'dado': 'dataset',
    'escala': 'scale',
    'sexo': 'gender'
}

dfs[dataset].rename(
    columns=migration_rules, inplace=True
)
dfs[dataset].dataset = dfs[dataset].dataset.map(dataset_id)
dfs[dataset].scale = dfs[dataset].scale.map(scale_id)
dfs[dataset].head()

0_4_years  10_19_years  2_4_years  20_29_years  30_39_years  40_49_years  \
0        0.0          0.0        NaN          0.0          0.0          0.0   
1        0.0          0.0        NaN          0.0          0.0          0.0   
2        0.0          0.0        NaN          0.0          0.0          0.0   
3        0.0          0.0        NaN          0.0          0.0          0.0   
4        0.0          0.0        NaN          0.0          0.0          0.0   

   5_9_years  50_59_years  60_years_or_more  lt_2_years  delayed  flu_a  \
0        0.0          0.0               0.0         NaN      0.0    0.0   
1        0.0          0.0               0.0         NaN      0.0    0.0   
2        0.0          0.0               0.0         NaN      0.0    0.0   
3        0.0          0.0               0.0         NaN      0.0    0.0   
4        0.0          0.0               0.0         NaN      0.0    0.0   

   flu_b  inconclusive  unknown_age  negative  not_tested  others  \
0    0.0           0.0          NaN       0.0         0.0     0.0   
1    0.0           0.0          NaN       0.0         0.0     0.0   
2    0.0           0.0          NaN       0.0         0.0     0.0   
3    0.0           0.0          NaN       0.0         0.0     0.0   
4    0.0           0.0          NaN       0.0         0.0     0.0   

   positive_cases  value situation  TESTING_IGNORED    type state_country  \
0             0.0    0.0    stable              0.0  Estado            11   
1             0.0    0.0    stable              0.0  Estado            11   
2             0.0    0.0    stable              0.0  Estado            11   
3             0.0    0.0    stable              0.0  Estado            11   
4             0.0    0.0    stable              0.0  Estado            11   

  state_country_name  vsr  dataset  epiweek  epiyear epiyearweek  scale gender  
0           Rondônia  0.0        1        1     2009     2009W01      1      F  
1           Rondônia  0.0        1        1     2009     2009W01      1      M  
2           Rondônia  0.0        1        1     2009     2009W01      1  Total  
3           Rondônia  0.0        1        2     2009     2009W02      1      F  
4           Rondônia  0.0        1        2     2009     2009W02      1      M

In [21]:
# primary_keys
pks[dataset] = ['dataset', 'scale', 'state_country', 'epiyear', 'epiweek']

dfs[dataset].set_index(pks[dataset], inplace=True)
dfs[dataset].head()

0_4_years  10_19_years  \
dataset scale state_country epiyear epiweek                           
1       1     11            2009    1              0.0          0.0   
                                    1              0.0          0.0   
                                    1              0.0          0.0   
                                    2              0.0          0.0   
                                    2              0.0          0.0   

                                             2_4_years  20_29_years  \
dataset scale state_country epiyear epiweek                           
1       1     11            2009    1              NaN          0.0   
                                    1              NaN          0.0   
                                    1              NaN          0.0   
                                    2              NaN          0.0   
                                    2              NaN          0.0   

                                             30_39_years  40_49_years  \
dataset scale state_country epiyear epiweek                             
1       1     11            2009    1                0.0          0.0   
                                    1                0.0          0.0   
                                    1                0.0          0.0   
                                    2                0.0          0.0   
                                    2                0.0          0.0   

                                             5_9_years  50_59_years  \
dataset scale state_country epiyear epiweek                           
1       1     11            2009    1              0.0          0.0   
                                    1              0.0          0.0   
                                    1              0.0          0.0   
                                    2              0.0          0.0   
                                    2              0.0          0.0   

                                             60_years_or_more  lt_2_years  \
dataset scale state_country epiyear epiweek                                 
1       1     11            2009    1                     0.0         NaN   
                                    1                     0.0         NaN   
                                    1                     0.0         NaN   
                                    2                     0.0         NaN   
                                    2                     0.0         NaN   

                                             delayed  flu_a  flu_b  \
dataset scale state_country epiyear epiweek                          
1       1     11            2009    1            0.0    0.0    0.0   
                                    1            0.0    0.0    0.0   
                                    1            0.0    0.0    0.0   
                                    2            0.0    0.0    0.0   
                                    2            0.0    0.0    0.0   

                                             inconclusive  unknown_age  \
dataset scale state_country epiyear epiweek                              
1       1     11            2009    1                 0.0          NaN   
                                    1                 0.0          NaN   
                                    1                 0.0          NaN   
                                    2                 0.0          NaN   
                                    2                 0.0          NaN   

                                             negative  not_tested  others  \
dataset scale state_country epiyear epiweek                                 
1       1     11            2009    1             0.0         0.0     0.0   
                                    1             0.0         0.0     0.0   
                                    1             0.0         0.0     0.0   
                                    2             0.0         0.0     0.0   
                           

## mem-report

In [22]:
dataset = 'mem-report'

In [23]:
dfs[dataset].head()

UF  População  \
0  11    1805788   
1  12     829619   
2  13    4063614   
3  14     522636   
4  15    8366628   

   Média geométrica do pico de infecção das temporadas regulares  \
0                                           0.803458               
1                                           0.953503               
2                                           0.119378               
3                                           0.473615               
4                                           0.256230               

   região de baixa atividade típica  limiar pré-epidêmico  intensidade alta  \
0                                 0              0.103162          0.863224   
1                                 0              0.685939          1.783885   
2                                 0              0.042171          0.165851   
3                                 0              0.414595          0.593964   
4                                 0              0.083865          0.466878   

   intensidade muito alta  SE típica do início do surto  \
0                1.168160                          12.0   
1                3.053363                          24.0   
2                0.212193                          12.0   
3                0.801757                          26.0   
4                0.859727                          12.0   

   SE típica do início do surto - IC inferior (2,5%)  \
0                                                9.0   
1                                                5.0   
2                                                4.0   
3                                               13.0   
4                                               10.0   

   SE típica do início do surto - IC superior (97,5%)  \
0                                               22.0    
1                                               49.0    
2                                               25.0    
3                                               50.0    
4                                               18.0    

   duração típica do surto  duração típica do surto - IC inferior (2,5%)  \
0                      8.0                                           4.0   
1                      6.0                                           3.0   
2                     12.0                                           6.0   
3                      6.0                                           3.0   
4                     10.0                                           5.0   

   duração típica do surto - IC superior (97,5%)  \
0                                           15.0   
1                                           16.0   
2                                           13.0   
3                                           10.0   
4                                           13.0   

  temporadas utilizadas para os corredores endêmicos   ano  \
0                             2010, 2013, 2014, 2016  2017   
1           2010, 2011, 2012, 2013, 2014, 2015, 2016  2017   
2                 2010, 2012, 2013, 2014, 2015, 2016  2017   
3                       2012, 2013, 2014, 2015, 2016  2017   
4                             2012, 2014, 2015, 2016  2017   

  Unidade da Federação    Tipo  dado      escala  
0             Rondônia  Estado  srag  incidência  
1                 Acre  Estado  srag  incidência  
2             Amazonas  Estado  srag  incidência  
3              Roraima  Estado  srag  incidência  
4                 Pará  Estado  srag  incidência

In [24]:
dfs[dataset].keys()

Index(['UF', 'População',
       'Média geométrica do pico de infecção das temporadas regulares',
       'região de baixa atividade típica', 'limiar pré-epidêmico',
       'intensidade alta', 'intensidade muito alta',
       'SE típica do início do surto',
       'SE típica do início do surto - IC inferior (2,5%)',
       'SE típica do início do surto - IC superior (97,5%)',
       'duração típica do surto',
       'duração típica do surto - IC inferior (2,5%)',
       'duração típica do surto - IC superior (97,5%)',
       'temporadas utilizadas para os corredores endêmicos', 'ano',
       'Unidade da Federação', 'Tipo', 'dado', 'escala'],
      dtype='object')

In [25]:
migration_rules = {
    'UF': 'state_country',
    'População': 'population',
    'Média geométrica do pico de infecção das temporadas regulares': 'geom_average_peak',
    'região de baixa atividade típica': 'low_activiy_region',
    'limiar pré-epidêmico': 'pre_epidemic_threshold',
    'intensidade alta': 'high_threshold',
    'intensidade muito alta': 'very_high_threshold',
    'SE típica do início do surto': 'epi_start',
    'SE típica do início do surto - IC inferior (2,5%)': 'epi_start_ci_lower',
    'SE típica do início do surto - IC superior (97,5%)': 'epi_start_ci_upper',
    'duração típica do surto': 'epi_duration',
    'duração típica do surto - IC inferior (2,5%)': 'epi_duration_ci_lower',
    'duração típica do surto - IC superior (97,5%)': 'epi_duration_ci_upper',
    'temporadas utilizadas para os corredores endêmicos': 'regular_seasons',
    'ano': 'year',
    'Unidade da Federação': 'state_country_name', # Not needed in the table
    'Tipo': 'type',
    'dado': 'dataset',
    'escala': 'scale'
}

dfs[dataset].rename(
    columns=migration_rules, inplace=True
)
dfs[dataset].dataset = dfs[dataset].dataset.map(dataset_id)
dfs[dataset].scale = dfs[dataset].scale.map(scale_id)
dfs[dataset].head()

state_country  population  geom_average_peak  low_activiy_region  \
0            11     1805788           0.803458                   0   
1            12      829619           0.953503                   0   
2            13     4063614           0.119378                   0   
3            14      522636           0.473615                   0   
4            15     8366628           0.256230                   0   

   pre_epidemic_threshold  high_threshold  very_high_threshold  epi_start  \
0                0.103162        0.863224             1.168160       12.0   
1                0.685939        1.783885             3.053363       24.0   
2                0.042171        0.165851             0.212193       12.0   
3                0.414595        0.593964             0.801757       26.0   
4                0.083865        0.466878             0.859727       12.0   

   epi_start_ci_lower  epi_start_ci_upper  epi_duration  \
0                 9.0                22.0           8.0   
1                 5.0                49.0           6.0   
2                 4.0                25.0          12.0   
3                13.0                50.0           6.0   
4                10.0                18.0          10.0   

   epi_duration_ci_lower  epi_duration_ci_upper  \
0                    4.0                   15.0   
1                    3.0                   16.0   
2                    6.0                   13.0   
3                    3.0                   10.0   
4                    5.0                   13.0   

                            regular_seasons  year state_country_name    type  \
0                    2010, 2013, 2014, 2016  2017           Rondônia  Estado   
1  2010, 2011, 2012, 2013, 2014, 2015, 2016  2017               Acre  Estado   
2        2010, 2012, 2013, 2014, 2015, 2016  2017           Amazonas  Estado   
3              2012, 2013, 2014, 2015, 2016  2017            Roraima  Estado   
4                    2012, 2014, 2015, 2016  2017               Pará  Estado   

   dataset  scale  
0        1      1  
1        1      1  
2        1      1  
3        1      1  
4        1      1

In [26]:
# primary_keys
pks[dataset] = ['dataset', 'scale', 'state_country', 'year']

dfs[dataset].set_index(pks[dataset], inplace=True)
dfs[dataset].head()

population  geom_average_peak  \
dataset scale state_country year                                  
1       1     11            2017     1805788           0.803458   
              12            2017      829619           0.953503   
              13            2017     4063614           0.119378   
              14            2017      522636           0.473615   
              15            2017     8366628           0.256230   

                                  low_activiy_region  pre_epidemic_threshold  \
dataset scale state_country year                                               
1       1     11            2017                   0                0.103162   
              12            2017                   0                0.685939   
              13            2017                   0                0.042171   
              14            2017                   0                0.414595   
              15            2017                   0                0.083865   

                                  high_threshold  very_high_threshold  \
dataset scale state_country year                                        
1       1     11            2017        0.863224             1.168160   
              12            2017        1.783885             3.053363   
              13            2017        0.165851             0.212193   
              14            2017        0.593964             0.801757   
              15            2017        0.466878             0.859727   

                                  epi_start  epi_start_ci_lower  \
dataset scale state_country year                                  
1       1     11            2017       12.0                 9.0   
              12            2017       24.0                 5.0   
              13            2017       12.0                 4.0   
              14            2017       26.0                13.0   
              15            2017       12.0                10.0   

                                  epi_start_ci_upper  epi_duration  \
dataset scale state_country year                                     
1       1     11            2017                22.0           8.0   
              12            2017                49.0           6.0   
              13            2017                25.0          12.0   
              14            2017                50.0           6.0   
              15            2017                18.0          10.0   

                                  epi_duration_ci_lower  \
dataset scale state_country year                          
1       1     11            2017                    4.0   
              12            2017                    3.0   
              13            2017                    6.0   
              14            2017                    3.0   
              15            2017                    5.0   

                                  epi_duration_ci_upper  \
dataset scale state_country year                          
1       1     11            2017                   15.0   
              12            2017                   16.0   
              13            2017                   13.0   
              14            2017                   10.0   
              15            2017                   13.0   

                                                           regular_seasons  \
dataset scale state_country year                                             
1       1     11            2017                    2010, 2013, 2014, 2016   
              12            2017  2010, 2011, 2012, 2013, 2014, 2015, 2016   
              13            2017        2010, 2012, 2013, 2014, 2015, 2016   
              14            2017              2012, 2013, 2014, 2015, 2016   
              15            2017                    2012, 2014, 2015, 2016   

                                 state_country_name    type  
dataset scale state_country year                             
1       1     11            201

## mem-typical

In [27]:
dataset = 'mem-typical'

In [28]:
dfs[dataset].head()

UF  População  epiweek  corredor baixo  corredor mediano  corredor alto  \
0  11    1805788        1             0.0          0.013703       0.041109   
1  11    1805788        2             0.0          0.000000       0.000000   
2  11    1805788        3             0.0          0.013703       0.041109   
3  11    1805788        4             0.0          0.028790       0.087644   
4  11    1805788        5             0.0          0.000000       0.000000   

    ano Unidade da Federação    Tipo  dado      escala  
0  2017             Rondônia  Estado  srag  incidência  
1  2017             Rondônia  Estado  srag  incidência  
2  2017             Rondônia  Estado  srag  incidência  
3  2017             Rondônia  Estado  srag  incidência  
4  2017             Rondônia  Estado  srag  incidência

In [29]:
dfs[dataset].keys()

Index(['UF', 'População', 'epiweek', 'corredor baixo', 'corredor mediano',
       'corredor alto', 'ano', 'Unidade da Federação', 'Tipo', 'dado',
       'escala'],
      dtype='object')

In [30]:
migration_rules = {
    'UF': 'state_country',
    'População': 'population',
    'corredor baixo': 'low',
    'corredor mediano': 'median',
    'corredor alto': 'high',
    'ano': 'year',
    'Unidade da Federação': 'state_country_name', # Not needed in the table
    'Tipo': 'type',
    'dado': 'dataset',
    'escala': 'scale'
}

dfs[dataset].rename(
    columns=migration_rules, inplace=True
)
dfs[dataset].dataset = dfs[dataset].dataset.map(dataset_id)
dfs[dataset].scale = dfs[dataset].scale.map(scale_id)
dfs[dataset].head()

state_country  population  epiweek  low    median      high  year  \
0            11     1805788        1  0.0  0.013703  0.041109  2017   
1            11     1805788        2  0.0  0.000000  0.000000  2017   
2            11     1805788        3  0.0  0.013703  0.041109  2017   
3            11     1805788        4  0.0  0.028790  0.087644  2017   
4            11     1805788        5  0.0  0.000000  0.000000  2017   

  state_country_name    type  dataset  scale  
0           Rondônia  Estado        1      1  
1           Rondônia  Estado        1      1  
2           Rondônia  Estado        1      1  
3           Rondônia  Estado        1      1  
4           Rondônia  Estado        1      1

In [31]:
# primary_keys
pks[dataset] = ['dataset', 'scale', 'state_country', 'year', 'epiweek']

dfs[dataset].set_index(pks[dataset], inplace=True)
dfs[dataset].head()

population  low    median      high  \
dataset scale state_country year epiweek                                        
1       1     11            2017 1           1805788  0.0  0.013703  0.041109   
                                 2           1805788  0.0  0.000000  0.000000   
                                 3           1805788  0.0  0.013703  0.041109   
                                 4           1805788  0.0  0.028790  0.087644   
                                 5           1805788  0.0  0.000000  0.000000   

                                         state_country_name    type  
dataset scale state_country year epiweek                             
1       1     11            2017 1                 Rondônia  Estado  
                                 2                 Rondônia  Estado  
                                 3                 Rondônia  Estado  
                                 4                 Rondônia  Estado  
                                 5                 Rondônia  Estado

@TODO convert regions id to numerical value